In [16]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse
import requests, re, time
import pandas_datareader
import pickle
import math

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import seaborn as sns 

import statsmodels.api as sm
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver
import time

import geopy.distance

In [37]:
path2gecko = '/Users/holger/Documents/Python/Harmsen_Repo/Metro-Study/Gecko/geckodriver'
browser = webdriver.Firefox(executable_path=path2gecko) # start the browser with a path to the geckodriver.

In [40]:
def WikiScraper(no_metros):
    url_base='https://da.wikipedia.org/wiki/Stationer_p%C3%A5_K%C3%B8benhavns_Metro'
    site = browser.get(url_base)
    html = site.text
    time.sleep(1)

    # Creating DataFrame and dictionary for information-storing:
    data = pd.DataFrame()
    dic_list = []

    for i in range(1,no_metros+1):
        link = browser.find_element_by_xpath(f'/html/body/div[3]/div[3]/div[5]/div/table[1]/tbody/tr[{i}]/td[1]/a')
        link.click()
        time.sleep(2)
        soup = BeautifulSoup(html,'lxml')
        Header = soup.title


        # READ INFORMATION
        # 1) Appending all information to DataFrame
        table = pd.read_html(browser.page_source)[0]
        data[f'Traits_{i}'] = table.iloc[:,0]
        data[f'{i}'] = table.iloc[:,1]
        data = pd.concat(data,['Header',Header],ignore_index=True)

        # 2) Creating a dictionary with all information:
        dic = {}
        for j in range(0,len(data[f'Traits_{i}'])):
            dic[str(data[f'Traits_{i}'][j])] = str(data[f'{i}'][j])

        dic_list.append(dic)

        # RETURN 
        browser.get(url_base)
        time.sleep(2)


    return [data, dic_list]

In [41]:
Metros = WikiScraper(1)

AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
Metros[1]

[{'nan': 'nan',
  'Overblik': 'Overblik',
  'Land': 'Danmark',
  'Adresse': 'Godthåbsvej 312000 Frederiksberg',
  'Kommune': 'Frederiksberg Kommune',
  'Jernbane': 'Cityringen',
  'Nabostationer': 'Nuuks Plads Station, Frederiksberg Station',
  'Forkortelse': 'Amh',
  'Afstand': '11,3 km fra Københavns Hovedbanegård (mod uret rundt)',
  'Åbningsdato': '29. september 2019[1]',
  'Takstzone': '1',
  'Perron(er)': '1',
  'Perronspor': '2',
  'Koordinater': '55°41′9″N 12°31′58″Ø\ufeff / \ufeff55.68583°N 12.53278°ØKoordinater: 55°41′9″N 12°31′58″Ø\ufeff / \ufeff55.68583°N 12.53278°Ø (kort)',
  'Transport': 'Transport',
  'Metro': 'nan',
  'Buslinjer': '2A',
  'Information med symbolet hentes fra Wikidata.\xa0[ redigér på Wikidata ]': 'Information med symbolet hentes fra Wikidata.\xa0[ redigér på Wikidata ]'}]